## Scraping Tweets

In [1]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns


import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re  
import spacy
from collections import Counter

In [2]:
nlp = spacy.load('es_core_news_sm') #en_core_news_lg

In [3]:
%run ./keys.ipynb

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
api.me().screen_name

'emiliomantilla6'

In [6]:
my_timeline = api.home_timeline()

#### Scraping User's timeline tweets (new version):

In [ ]:
# Code from https://github.com/codingforentrepreneurs/30-Days-of-Python/blob/master/tutorial-reference/Day%2021/3%20-%20My%20Timeline%20%26%20User%20Timeline.ipynb

#def extract_timeline_as_df(timeline_list):
#    columns = set()
#    allowed_types = [str, int]
#    tweets_data = []
#    for status in timeline_list:
#        status_dict = dict(vars(status))
#        keys = status_dict.keys()
#        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
#        for k in keys:
#            try:
#                v_type = type(status_dict[k])
#            except:
#                v_type = None
#            if v_type != None:
#                if v_type in allowed_types:
#                    single_tweet_data[k] = status_dict[k]
#                    columns.add(k)
#        tweets_data.append(single_tweet_data)


#    header_cols = list(columns)
#    header_cols.append("user")
#    header_cols.append('author')
#    df = pd.DataFrame(tweets_data, columns=header_cols)
#    return df

In [ ]:
#user = api.get_user("MashiRafael")
#user_timeline = user.timeline()
#df3 = extract_timeline_as_df(user_timeline)
#df3

#### Scraping User's timeline tweets (old version):

In [7]:
cursor = tweepy.Cursor(api.user_timeline,id="MashiRafael",tweet_mode="extended").items(1)

In [8]:
#for i in cursor: #Not very informative
#    print(i)
    
for i in cursor: # To see available things you could extract from cursor object. e.g. text
    print(dir(i))

#for i in cursor:
#    print(i.retweet_count)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'display_text_range', 'entities', 'favorite', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'retweet', 'retweet_count', 'retweeted', 'retweeted_status', 'retweets', 'source', 'source_url', 'truncated', 'user']


In [9]:
# Taken from this link:
# https://www.youtube.com/watch?v=bNDRiaFyLrs

number_of_tweets =4000 #Change accordingly to pull latest number of tweets. Limit seems to be 3240
user = []
author = []
id_ = []
id_str = []
tweets = []
language = []
source = []
source_url = []
retweet_count = []
likes = []
time = []

for i in tweepy.Cursor(api.user_timeline, id="MashiRafael", tweet_mode="extended").items(number_of_tweets): #"extended"
    #tweets.append(i.text)
    user.append(i.user.screen_name)
    author.append(i.author.screen_name)
    id_.append(i.id)
    id_str.append(i.id_str)
    tweets.append(i.full_text)
    language.append(i.lang)
    source.append(i.source)
    source_url.append(i.source_url)
    retweet_count.append(i.retweet_count)
    likes.append(i.favorite_count)
    time.append(i.created_at)

In [10]:
df = pd.DataFrame({'user':user,'author':author,'id':id_,'id_str':id_str,
                   'tweets':tweets,'language':language,'source':source,'source_url':source_url,
                   'retweet_count':retweet_count,'likes':likes,'time':time})

In [11]:
df

,user,author,id,id_str,tweets,language,source,source_url,retweet_count,likes,time
0,MashiRafael,MashiRafael,1486469645897019400,1486469645897019400,RT @relicheandres: BANCOCRACIA \n\n287 mil (do...,es,Twitter for iPhone,http://twitter.com/download/iphone,177,0,2022-01-26 22:42:35
1,MashiRafael,MashiRafael,1486433624866140163,1486433624866140163,https://t.co/Qii4tz3VZd,und,Twitter for iPhone,http://twitter.com/download/iphone,236,456,2022-01-26 20:19:27
2,MashiRafael,MashiRafael,1486416582914060290,1486416582914060290,RT @MashiRafael: 🇪🇨🇪🇨🇪🇨\nNo fue casualidad. Fu...,es,Twitter for iPhone,http://twitter.com/download/iphone,439,0,2022-01-26 19:11:44
3,MashiRafael,MashiRafael,1486415886609170436,1486415886609170436,RT @MashiRafael: 🇪🇨🇪🇨🇪🇨\nEcuatorianos:\nTomen ...,es,Twitter for iPhone,http://twitter.com/download/iphone,1314,0,2022-01-26 19:08:58
4,MashiRafael,MashiRafael,1486413954737967105,1486413954737967105,"🇪🇨🇪🇨🇪🇨\nEsta es la verdad histórica, con DATOS...",es,Twitter for iPhone,http://twitter.com/download/iphone,188,293,2022-01-26 19:01:17
...,...,...,...,...,...,...,...,...,...,...,...
3237,MashiRafael,MashiRafael,1454861081936023556,1454861081936023556,Más humillación para el país por la evidente p...,es,Twitter for iPhone,http://twitter.com/download/iphone,335,563,2021-10-31 17:21:26
3238,MashiRafael,MashiRafael,1454857627620585476,1454857627620585476,🇪🇨🇪🇨🇪🇨\nEsto es EXTREMADAMENTE grave. Las rese...,es,Twitter for iPhone,http://twitter.com/download/iphone,406,612,2021-10-31 17:07:43
3239,MashiRafael,MashiRafael,1454857080859414533,1454857080859414533,🇪🇨🇪🇨🇪🇨\nSi no están obligados a regresar al pa...,es,Twitter for iPhone,http://twitter.com/download/iphone,638,1175,2021-10-31 17:05:32
3240,MashiRafael,MashiRafael,1454855989975556099,1454855989975556099,"En la Sainte Chapelle de París, en mi avión pr...",es,Twitter for iPhone,http://twitter.com/download/iphone,655,2533,2021-10-31 17:01:12


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3242 entries, 0 to 3241
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user           3242 non-null   object        
 1   author         3242 non-null   object        
 2   id             3242 non-null   int64         
 3   id_str         3242 non-null   object        
 4   tweets         3242 non-null   object        
 5   language       3242 non-null   object        
 6   source         3242 non-null   object        
 7   source_url     3242 non-null   object        
 8   retweet_count  3242 non-null   int64         
 9   likes          3242 non-null   int64         
 10  time           3242 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 278.7+ KB


In [13]:
df['Hour'] = df['time'].dt.hour
df['Date'] = df['time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])

In [14]:
#df.to_csv("RC_Tweets_02242021_05142021.csv")
#df.to_csv("RC_Tweets_05142021_05172021.csv")
#df.to_csv("RC_Tweets_03012021_05282021.csv")
#df.to_csv("RC_Tweets_05292021_06052021.csv")
#df.to_csv("RC_Tweets_06052021_06222021.csv")
df.to_csv("RC_Tweets_103121_012622.csv")